My analysis idea is using different variables to predict whether the player will be within the first three ranks.

In [0]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from pyspark.sql import SparkSession

results_path = "s3://columbia-gr5069-main/raw/results.csv"
races_path = "s3://columbia-gr5069-main/raw/races.csv"

results = spark.read.csv(results_path, header=True, inferSchema=True)
races = spark.read.csv(races_path, header=True, inferSchema=True)

display(results.limit(5))
display(races.limit(5))

resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [0]:
df = results.join(races, on="raceId").filter("positionOrder IS NOT NULL")
df = df.select("resultId", "driverId", "constructorId", "grid", "positionOrder", "circuitId", "year")
df = df.withColumn("label", (df["positionOrder"] <= 3).cast("int"))
pdf = df.toPandas()
pdf = pdf.dropna()

X = pdf[["driverId", "constructorId", "grid", "circuitId", "year"]]
y = pdf["label"]

X = pd.get_dummies(X.astype(str))

1. Create two (2) new tables in your own fatabse where you'll store the predictions from each model for this exercise.

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS f1_user_db")
spark.sql("""
  CREATE OR REPLACE TABLE f1_user_db.predictions_logreg (
    id INT,
    prediction INT
  )
""")

spark.sql("""
  CREATE OR REPLACE TABLE f1_user_db.predictions_rf (
    id INT,
    prediction INT
  )
""")

DataFrame[]

 2. Build two (2) predictive models using MLflow, logging hyperparameters, the model itself, four metrics, and two artifcats. Submit submit your MLflow experiments as part of your assignments

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def train_model(model, name, table_name):
    with mlflow.start_run(run_name=name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)

        mlflow.log_param("model_type", type(model).__name__)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1", f1)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)

        mlflow.sklearn.log_model(model, "model")

        with open("/tmp/features.txt", "w") as f:
            f.write("\n".join(X.columns))
        mlflow.log_artifact("/tmp/features.txt")

        np.savetxt("/tmp/predictions.csv", y_pred, fmt="%d")
        mlflow.log_artifact("/tmp/predictions.csv")

        spark.sql(f"DROP TABLE IF EXISTS f1_user_db.{table_name}")

        pred_df = pd.DataFrame({
            "id": X_test.index.astype("int32"),
            "prediction": y_pred.astype("int32")
        })

        spark_df = spark.createDataFrame(pred_df)
        spark_df.write.mode("overwrite").saveAsTable(f"f1_user_db.{table_name}")
        print(f"{name} completed. F1 Score = {f1:.4f}")


3. For each model, store its predictions in the corresponding table you created in your own database. Ensure you are using your own database to store your predictions.

In [0]:
train_model(LogisticRegression(max_iter=1000), "LogReg_Model", "predictions_logreg")
train_model(RandomForestClassifier(n_estimators=100), "RandomForest_Model", "predictions_rf")


2025/04/22 15:21:08 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


LogReg_Model completed. F1 Score = 0.5087


2025/04/22 15:21:48 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


RandomForest_Model completed. F1 Score = 0.4563
